# Week 8 Homework

***Due (pushed to your GitHub branch) on 11/8 by 11:59 pm***

## Mark duplicates

Use Picard Tools to mark the duplicates of the HISAT and STAR alignments. Store the logs in a new directory.

In [1]:
%mkdir logs_lemons
%mkdir dedup_lemons

!picard MarkDuplicates -I ../6_alignment/alignment_lemons/star/Aligned.sortedByCoord.out.bam -M logs_lemons/star_duplicates -O dedup_lemons/star.bam --VALIDATION_STRINGENCY SILENT

!picard MarkDuplicates -I ../6_alignment/alignment_lemons/hisat/merged.bam -M logs_lemons/hisat_duplicates -O dedup_lemons/hisat.bam --VALIDATION_STRINGENCY SILENT

[0.001s][warning][os,container] Duplicate cpuset controllers detected. Picking /sys/fs/cgroup/cpuset, skipping /dev/cpuset.
20:34:17.433 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/data/users/corwinbm5021/.conda/envs/biol343_20241029/share/picard-3.2.0-0/picard.jar!/com/intel/gkl/native/libgkl_compression.so
[Fri Nov 01 20:34:17 CDT 2024] MarkDuplicates --INPUT ../6_alignment/alignment_lemons/star/Aligned.sortedByCoord.out.bam --OUTPUT dedup_lemons/star.bam --METRICS_FILE logs_lemons/star_duplicates --VALIDATION_STRINGENCY SILENT --MAX_SEQUENCES_FOR_DISK_READ_ENDS_MAP 50000 --MAX_FILE_HANDLES_FOR_READ_ENDS_MAP 8000 --SORTING_COLLECTION_SIZE_RATIO 0.25 --TAG_DUPLICATE_SET_MEMBERS false --REMOVE_SEQUENCING_DUPLICATES false --TAGGING_POLICY DontTag --CLEAR_DT true --DUPLEX_UMI false --FLOW_MODE false --FLOW_DUP_STRATEGY FLOW_QUALITY_SUM_STRATEGY --USE_END_IN_UNPAIRED_READS false --USE_UNPAIRED_CLIPPED_END false --UNPAIRED_END_UNCERTAINTY 0 --UNPAIRED_START_UNC

## Counting

Use featureCounts to count reads. You may need to explicitly set the features and meta-features with `-t` and `-g`.

In [2]:
!featureCounts -T 32 \
    dedup_lemons/star.bam \
    -T 32 \
    --byReadGroup \
    -s 1 \
    --ignoreDup \
    -M \
    --fraction \
    -a ../2_genome_exploration/lemons/lemon_annotations.gtf \
    -t "CDS" \
    -o star_counts_lemons.tsv \
    --verbose

!featureCounts -T 32 \
    dedup_lemons/hisat.bam \
    -T 32 \
    --byReadGroup \
    -s 1 \
    --ignoreDup \
    -M \
    --fraction \
    -a ../2_genome_exploration/lemons/lemon_annotations.gtf \
    -t "CDS" \
    -o hisat_counts_lemons.tsv \
    --verbose


        ==========     _____ _    _ ____  _____  ______          _____  
        =====         / ____| |  | |  _ \|  __ \|  ____|   /\   |  __ \ 
          =====      | (___ | |  | | |_) | |__) | |__     /  \  | |  | |
            ====      \___ \| |  | |  _ <|  _  /|  __|   / /\ \ | |  | |
              ====    ____) | |__| | |_) | | \ \| |____ / ____ \| |__| |
        ==========   |_____/ \____/|____/|_|  \_\______/_/    \_\_____/
	  v2.0.6

//========================== featureCounts setting ===========================\\
||                                                                            ||
||             Input files : 1 BAM file                                       ||
||                                                                            ||
||                           star.bam                                         ||
||                                                                            ||
||             Output file : star_counts_lemons.tsv              

## Report

Use MultiQC to aggregate a report that incorporates all steps of the pipeline.

In [2]:
!multiqc --force -d ../5_fastq/fastq_lemons/qc/ ../5_fastq/trimmed_lemons/qc/ ../6_alignment/alignment_lemons ../7_alignment_qc . -n multiqc_report_lemons.html


  /// ]8;id=630273;https://multiqc.info\MultiQC]8;;\ 🔍 | v1.17

|           multiqc | Prepending directory to sample names
|           multiqc | Search path : /data/users/corwinbm5021/BIOL343/5_fastq/fastq_lemons/qc
|           multiqc | Search path : /data/users/corwinbm5021/BIOL343/5_fastq/trimmed_lemons/qc
|           multiqc | Search path : /data/users/corwinbm5021/BIOL343/6_alignment/alignment_lemons
|           multiqc | Search path : /data/users/corwinbm5021/BIOL343/7_alignment_qc
|           multiqc | Search path : /data/users/corwinbm5021/BIOL343/8_counting
|         searching | ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 418/418  gs_lemons/star_duplicatess.tsv
|    feature_counts | Found 2 reports
|            picard | Found 2 MarkDuplicates reports
|          samtools | Found 1 stats reports
|          samtools | Found 2 flagstat reports
|              star | Found 1 reports
|            hisat2 | Found 55 reports
|            fastqc | Found 108 reports
|           mul

## Snakemake Pipeline (optional)

1. Copy your `Snakefile` and `config.yaml` from `7_alignment_qc` to `8_counting`.  
2. In the `Snakefile`, add the following rules:  
    a. `mark_duplicates` - use Picard Tools to mark the duplicate reads in the STAR and HISAT alignments.  
    b. `count` - use featureCounts to count reads in the alignments with duplicates marked.  
    c. `report` - use `multiqc` to aggregate all QC data into a single report. This should be an update of the report from the previous week's homework.  

This `Snakefile` should not use any `wrappers` because we already have the relevant software installed in the `biol343` conda environment. The pipeline should run to completion when any instructor or classmate runs `snakemake --use-conda`.